# 3. Image-Similar-FCNN-Binary
For landmark-recognition-2019 algorithm validation

## Run name

In [1]:
import time

project_name = 'Dog-Breed'
step_name = '3-Image-Similar-FCNN-Binary'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Dog-Breed_3-Image-Similar-FCNN-Binary_20190507-211416


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

cpu_amount:  4


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [4]:
import keras
from keras.utils import Sequence
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras.optimizers import *
from keras.regularizers import *
from keras.preprocessing.image import *
from keras.applications.inception_v3 import preprocess_input

Using TensorFlow backend.


## Project folders

In [5]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

vgg16_feature_file = os.path.join(feature_folder, 'feature_wrapper_171023.h5')
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')
print(vgg16_feature_file)
print(train_csv_file)
print(test_csv_file)
print(sample_submission_folder)

D:\Kaggle\dog-breed-identification\feature\feature_wrapper_171023.h5
D:\Kaggle\dog-breed-identification\input\train.csv
D:\Kaggle\dog-breed-identification\input\test.csv
D:\Kaggle\dog-breed-identification\input\sample_submission.csv


## Load feature

In [6]:
with h5py.File(vgg16_feature_file, 'r') as h:
    x_train = np.array(h['train'])
    y_train = np.array(h['train_label'])
    x_val = np.array(h['val'])
    y_val = np.array(h['val_label'])


In [7]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(9199, 512)
(9199,)
(1023, 512)
(1023,)


In [8]:
import random
random.choice(list(range(10)))
import copy
a = list(range(10, 20))
print(a)
a.remove(13)
print(a)

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[10, 11, 12, 14, 15, 16, 17, 18, 19]


## ImageSequence

In [9]:
# class ImageSequence(Sequence):
#     def __init__(self, x, y, batch_size, times_for_1_image, positive_rate):
#         self.x = x
#         self.y = y
#         self.batch_size = batch_size
#         self.times_for_1_image = times_for_1_image
#         self.positive_rate = positive_rate
        
#         self.len_x = self.x.shape[0]
#         self.index = list(range(self.len_x))
#         self.group = {}
#         self.classes = list(set(self.y))
#         self.classes.sort()
#         for c in self.classes:
#             self.group[c] = []
#         for i, y_i in enumerate(self.y):
#             temp_arr = self.group[y_i]
#             temp_arr.append(i)
#             self.group[y_i] = temp_arr
        
#     def __len__(self):
#         # times_for_1_image: the times to train one image
#         # 2: positive example and negative example
#         return self.times_for_1_image * 2 * (math.ceil(self.len_x/self.batch_size))
#     def __getitem__(self, idx):
#         batch_main_x = []
#         batch_libary_x = []
#         batch_x = {}
#         batch_y = [] # 0 or 1
#         for i in range(self.batch_size):
#             # prepare main image
#             item_main_image_idx = random.choice(self.index) # random choice one image from all train images
#             item_main_image_y = self.y[item_main_image_idx]
            
#             # prepare libary image
#             is_positive = random.random() < self.positive_rate
#             if is_positive: # chioce a positive image as libary_x
#                 # choice one image from itself group
#                 item_libary_image_idx = random.choice(self.group[item_main_image_y]) # don't exclude item_main_image_idx, so it could choice a idx same to item_main_image_idx.
#             else: # chioce a negative image as libary_x
#                 # choice group
#                 new_class = copy.deepcopy(self.classes)
#                 new_class.remove(item_main_image_y)
#                 item_libary_image_group_num = random.choice(new_class)
#                 # choice one image from group
#                 item_libary_image_idx = random.choice(self.group[item_libary_image_group_num])
#             # add item data to batch
#             batch_main_x.append(self.x[item_main_image_idx])
#             batch_libary_x.append(self.x[item_libary_image_idx])
#             batch_y.append(int(is_positive))
#         # concatenate array to np.array
#         batch_x = {
#             'main_input': np.array(batch_main_x),
#             'library_input': np.array(batch_libary_x)
#         }
#         batch_y = np.array(batch_y)
#         return batch_x, batch_y

# demo_sequence = ImageSequence(x_train[:200], y_train[:200], 128, 3, 0.1)
# print(len(demo_sequence))
# print(type(demo_sequence))

# batch_index = 0
# demo_batch = demo_sequence[batch_index]
# demo_batch_x = demo_batch[0]
# demo_batch_y = demo_batch[1]
# print(type(demo_batch_x))
# print(type(demo_batch_y))

# demo_main_input = demo_batch_x['main_input']
# demo_library_input = demo_batch_x['library_input']
# print(demo_main_input.shape)
# print(demo_library_input.shape)
# print(demo_batch_y.shape)

# # print(demo_main_input[0])
# print(demo_batch_y)

In [10]:
class ImageSequence(Sequence):
    def __init__(self, x, y, batch_size, times_for_1_image, positive_rate):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.times_for_1_image = times_for_1_image
        self.positive_rate = positive_rate
        
        self.len_x = self.x.shape[0]
        self.index = list(range(self.len_x))
        self.group = {}
        self.classes = list(set(self.y))
        self.classes.sort()
        for c in self.classes:
            self.group[c] = []
        for i, y_i in enumerate(self.y):
            temp_arr = self.group[y_i]
            temp_arr.append(i)
            self.group[y_i] = temp_arr
        
    def __len__(self):
        # times_for_1_image: the times to train one image
        # 2: positive example and negative example
        return self.times_for_1_image * 2 * (math.ceil(self.len_x/self.batch_size))
    def __getitem__(self, idx):
        batch_main_x = np.zeros((self.batch_size, self.x.shape[1]))
        batch_libary_x = np.zeros((self.batch_size, self.x.shape[1]))
        batch_x = {}
        batch_y = [] # 0 or 1
        for i in range(self.batch_size):
            # prepare main image
            item_main_image_idx = random.choice(self.index) # random choice one image from all train images
            item_main_image_y = self.y[item_main_image_idx]
            
            # prepare libary image
            is_positive = random.random() < self.positive_rate
            if is_positive: # chioce a positive image as libary_x
                # choice one image from itself group
                item_libary_image_idx = random.choice(self.group[item_main_image_y]) # don't exclude item_main_image_idx, so it could choice a idx same to item_main_image_idx.
            else: # chioce a negative image as libary_x
                # choice group
                new_class = copy.deepcopy(self.classes)
                new_class.remove(item_main_image_y)
                item_libary_image_group_num = random.choice(new_class)
                # choice one image from group
                item_libary_image_idx = random.choice(self.group[item_libary_image_group_num])
            # add item data to batch
            batch_main_x[i] = self.x[item_main_image_idx]
            batch_libary_x[i] = self.x[item_libary_image_idx]
            batch_y.append(int(is_positive))
        # concatenate array to np.array
        batch_x = {
            'main_input': batch_main_x,
            'library_input': batch_libary_x
        }
        batch_y = np.array(batch_y)
        return batch_x, batch_y

demo_sequence = ImageSequence(x_train[:200], y_train[:200], 128, 3, 0.1)
print(len(demo_sequence))
print(type(demo_sequence))

batch_index = 0
demo_batch = demo_sequence[batch_index]
demo_batch_x = demo_batch[0]
demo_batch_y = demo_batch[1]
print(type(demo_batch_x))
print(type(demo_batch_y))

demo_main_input = demo_batch_x['main_input']
demo_library_input = demo_batch_x['library_input']
print(demo_main_input.shape)
print(demo_library_input.shape)
print(demo_batch_y.shape)

# print(demo_main_input[0])
print(demo_batch_y)

12
<class '__main__.ImageSequence'>
<class 'dict'>
<class 'numpy.ndarray'>
(128, 512)
(128, 512)
(128,)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0]


In [11]:
train_sequence = ImageSequence(x_train, y_train, 32, 3, 0.5)
val_sequence = ImageSequence(x_val, y_val, 32, 3, 0.5)

## Model

In [12]:
main_input = Input((x_train.shape[1],), dtype='float32', name='main_input')
library_input = Input((x_train.shape[1],), dtype='float32', name='library_input')

x = keras.layers.concatenate([main_input, library_input])
x = Dense(x_train.shape[1]*2, activation='sigmoid')(x)

x = Dense(1024, activation='sigmoid')(x)
x = Dense(1024, activation='sigmoid')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=[main_input, library_input], outputs=[output])
model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 512)          0                                            
__________________________________________________________________________________________________
library_input (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 1024)         0           main_input[0][0]                 
                                                                 library_input[0][0]              
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         1049600     concatenate_1[0][0]              
__________

In [14]:
hist = model.fit_generator(
    train_sequence, 
    steps_per_epoch=128, 
    epochs=300, 
    verbose=1, 
    callbacks=None, 
    validation_data=val_sequence, 
    validation_steps=128, 
    class_weight=None, 
    max_queue_size=10, 
    workers=1, 
    use_multiprocessing=False, 
    shuffle=True, 
    initial_epoch=0
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/300
128/128 [==============================] - ETA: 2:43 - loss: 0.7173 - acc: 0.437 - ETA: 1:27 - loss: 0.7095 - acc: 0.484 - ETA: 1:02 - loss: 0.6981 - acc: 0.520 - ETA: 49s - loss: 0.7058 - acc: 0.515 - ETA: 41s - loss: 0.7066 - acc: 0.50 - ETA: 36s - loss: 0.7042 - acc: 0.51 - ETA: 32s - loss: 0.7079 - acc: 0.50 - ETA: 30s - loss: 0.7043 - acc: 0.51 - ETA: 28s - loss: 0.7039 - acc: 0.52 - ETA: 26s - loss: 0.7075 - acc: 0.52 - ETA: 24s - loss: 0.7068 - acc: 0.52 - ETA: 23s - loss: 0.7026 - acc: 0.53 - ETA: 22s - loss: 0.7027 - acc: 0.52 - ETA: 21s - loss: 0.7017 - acc: 0.52 - ETA: 20s - loss: 0.7013 - acc: 0.51 - ETA: 19s - loss: 0.7007 - acc: 0.51 - ETA: 19s - loss: 0.7004 - acc: 0.50 - ETA: 18s - loss: 0.7001 - acc: 0.50 - ETA: 18s - loss: 0.6996 - acc: 0.50 - ETA: 17s - loss: 0.6988 - acc: 0.51 - ETA: 17s - loss: 0.6981 - acc: 0.51 - ETA: 16s - loss: 0.6983 - acc: 0.51 - ETA: 16s - loss: 0.6973 - acc: 0.51 - ETA: 15s - loss:

128/128 [==============================] - ETA: 11s - loss: 0.6981 - acc: 0.46 - ETA: 11s - loss: 0.6967 - acc: 0.48 - ETA: 11s - loss: 0.6911 - acc: 0.55 - ETA: 12s - loss: 0.6991 - acc: 0.50 - ETA: 11s - loss: 0.6979 - acc: 0.50 - ETA: 11s - loss: 0.6953 - acc: 0.51 - ETA: 11s - loss: 0.6900 - acc: 0.54 - ETA: 11s - loss: 0.6925 - acc: 0.53 - ETA: 11s - loss: 0.6860 - acc: 0.54 - ETA: 11s - loss: 0.6893 - acc: 0.54 - ETA: 11s - loss: 0.6839 - acc: 0.55 - ETA: 11s - loss: 0.6874 - acc: 0.55 - ETA: 11s - loss: 0.6874 - acc: 0.55 - ETA: 11s - loss: 0.6802 - acc: 0.57 - ETA: 11s - loss: 0.6867 - acc: 0.56 - ETA: 11s - loss: 0.6890 - acc: 0.55 - ETA: 10s - loss: 0.6885 - acc: 0.55 - ETA: 10s - loss: 0.6894 - acc: 0.55 - ETA: 10s - loss: 0.6892 - acc: 0.55 - ETA: 10s - loss: 0.6909 - acc: 0.54 - ETA: 10s - loss: 0.6958 - acc: 0.53 - ETA: 10s - loss: 0.6980 - acc: 0.52 - ETA: 10s - loss: 0.6978 - acc: 0.52 - ETA: 10s - loss: 0.6978 - acc: 0.52 - ETA: 10s - loss: 0.6978 - acc: 0.52 - ETA: 10

128/128 [==============================] - ETA: 12s - loss: 0.7462 - acc: 0.34 - ETA: 12s - loss: 0.7208 - acc: 0.42 - ETA: 12s - loss: 0.7093 - acc: 0.46 - ETA: 12s - loss: 0.7025 - acc: 0.48 - ETA: 12s - loss: 0.7020 - acc: 0.49 - ETA: 12s - loss: 0.6991 - acc: 0.50 - ETA: 12s - loss: 0.6982 - acc: 0.50 - ETA: 13s - loss: 0.6978 - acc: 0.50 - ETA: 13s - loss: 0.6963 - acc: 0.52 - ETA: 13s - loss: 0.6939 - acc: 0.54 - ETA: 12s - loss: 0.6928 - acc: 0.53 - ETA: 12s - loss: 0.6921 - acc: 0.54 - ETA: 12s - loss: 0.6925 - acc: 0.54 - ETA: 12s - loss: 0.6913 - acc: 0.54 - ETA: 12s - loss: 0.6900 - acc: 0.55 - ETA: 12s - loss: 0.6918 - acc: 0.54 - ETA: 12s - loss: 0.6929 - acc: 0.53 - ETA: 11s - loss: 0.6917 - acc: 0.53 - ETA: 12s - loss: 0.6906 - acc: 0.54 - ETA: 11s - loss: 0.6905 - acc: 0.53 - ETA: 11s - loss: 0.6907 - acc: 0.53 - ETA: 11s - loss: 0.6903 - acc: 0.54 - ETA: 11s - loss: 0.6896 - acc: 0.54 - ETA: 11s - loss: 0.6893 - acc: 0.54 - ETA: 11s - loss: 0.6896 - acc: 0.54 - ETA: 11

128/128 [==============================] - ETA: 15s - loss: 0.7189 - acc: 0.50 - ETA: 14s - loss: 0.7262 - acc: 0.48 - ETA: 15s - loss: 0.7239 - acc: 0.48 - ETA: 14s - loss: 0.7178 - acc: 0.50 - ETA: 14s - loss: 0.7162 - acc: 0.48 - ETA: 13s - loss: 0.7099 - acc: 0.48 - ETA: 13s - loss: 0.7044 - acc: 0.49 - ETA: 12s - loss: 0.7000 - acc: 0.50 - ETA: 12s - loss: 0.6986 - acc: 0.50 - ETA: 12s - loss: 0.7113 - acc: 0.48 - ETA: 12s - loss: 0.7078 - acc: 0.49 - ETA: 12s - loss: 0.7064 - acc: 0.49 - ETA: 11s - loss: 0.7011 - acc: 0.51 - ETA: 11s - loss: 0.7002 - acc: 0.51 - ETA: 11s - loss: 0.6977 - acc: 0.52 - ETA: 11s - loss: 0.6979 - acc: 0.51 - ETA: 11s - loss: 0.6956 - acc: 0.52 - ETA: 11s - loss: 0.6944 - acc: 0.53 - ETA: 11s - loss: 0.6944 - acc: 0.53 - ETA: 11s - loss: 0.6928 - acc: 0.54 - ETA: 11s - loss: 0.6916 - acc: 0.55 - ETA: 11s - loss: 0.6923 - acc: 0.55 - ETA: 11s - loss: 0.6910 - acc: 0.56 - ETA: 10s - loss: 0.6910 - acc: 0.55 - ETA: 10s - loss: 0.6913 - acc: 0.55 - ETA: 10

128/128 [==============================] - ETA: 13s - loss: 0.6299 - acc: 0.75 - ETA: 12s - loss: 0.6772 - acc: 0.59 - ETA: 12s - loss: 0.6511 - acc: 0.61 - ETA: 12s - loss: 0.6668 - acc: 0.58 - ETA: 11s - loss: 0.6722 - acc: 0.56 - ETA: 11s - loss: 0.6771 - acc: 0.56 - ETA: 11s - loss: 0.6665 - acc: 0.57 - ETA: 11s - loss: 0.6713 - acc: 0.55 - ETA: 11s - loss: 0.6680 - acc: 0.56 - ETA: 11s - loss: 0.6669 - acc: 0.57 - ETA: 10s - loss: 0.6670 - acc: 0.57 - ETA: 10s - loss: 0.6603 - acc: 0.58 - ETA: 10s - loss: 0.6599 - acc: 0.58 - ETA: 10s - loss: 0.6613 - acc: 0.58 - ETA: 10s - loss: 0.6655 - acc: 0.57 - ETA: 10s - loss: 0.6632 - acc: 0.57 - ETA: 10s - loss: 0.6626 - acc: 0.57 - ETA: 10s - loss: 0.6607 - acc: 0.58 - ETA: 10s - loss: 0.6593 - acc: 0.59 - ETA: 10s - loss: 0.6585 - acc: 0.59 - ETA: 10s - loss: 0.6553 - acc: 0.60 - ETA: 10s - loss: 0.6563 - acc: 0.59 - ETA: 10s - loss: 0.6581 - acc: 0.59 - ETA: 9s - loss: 0.6588 - acc: 0.5924 - ETA: 9s - loss: 0.6573 - acc: 0.592 - ETA: 9

...

128/128 [==============================] - ETA: 11s - loss: 0.4871 - acc: 0.78 - ETA: 10s - loss: 0.4606 - acc: 0.81 - ETA: 10s - loss: 0.4488 - acc: 0.81 - ETA: 10s - loss: 0.4856 - acc: 0.78 - ETA: 10s - loss: 0.4808 - acc: 0.79 - ETA: 10s - loss: 0.4773 - acc: 0.78 - ETA: 10s - loss: 0.4693 - acc: 0.78 - ETA: 10s - loss: 0.4609 - acc: 0.78 - ETA: 10s - loss: 0.4688 - acc: 0.77 - ETA: 10s - loss: 0.4711 - acc: 0.77 - ETA: 10s - loss: 0.4629 - acc: 0.78 - ETA: 10s - loss: 0.4701 - acc: 0.77 - ETA: 10s - loss: 0.4758 - acc: 0.77 - ETA: 10s - loss: 0.4723 - acc: 0.77 - ETA: 10s - loss: 0.4768 - acc: 0.77 - ETA: 10s - loss: 0.4849 - acc: 0.76 - ETA: 10s - loss: 0.4901 - acc: 0.76 - ETA: 10s - loss: 0.4976 - acc: 0.76 - ETA: 10s - loss: 0.4970 - acc: 0.75 - ETA: 10s - loss: 0.5047 - acc: 0.75 - ETA: 10s - loss: 0.5076 - acc: 0.74 - ETA: 9s - loss: 0.5141 - acc: 0.7386 - ETA: 9s - loss: 0.5165 - acc: 0.740 - ETA: 9s - loss: 0.5130 - acc: 0.742 - ETA: 9s - loss: 0.5137 - acc: 0.741 - ETA: 9

128/128 [==============================] - ETA: 13s - loss: 0.4689 - acc: 0.81 - ETA: 15s - loss: 0.4541 - acc: 0.78 - ETA: 15s - loss: 0.4422 - acc: 0.78 - ETA: 14s - loss: 0.4551 - acc: 0.78 - ETA: 14s - loss: 0.4650 - acc: 0.77 - ETA: 13s - loss: 0.4647 - acc: 0.77 - ETA: 12s - loss: 0.4632 - acc: 0.77 - ETA: 12s - loss: 0.4563 - acc: 0.77 - ETA: 12s - loss: 0.4612 - acc: 0.77 - ETA: 12s - loss: 0.4587 - acc: 0.77 - ETA: 11s - loss: 0.4602 - acc: 0.77 - ETA: 11s - loss: 0.4642 - acc: 0.77 - ETA: 11s - loss: 0.4659 - acc: 0.77 - ETA: 11s - loss: 0.4712 - acc: 0.77 - ETA: 11s - loss: 0.4749 - acc: 0.76 - ETA: 10s - loss: 0.4682 - acc: 0.76 - ETA: 10s - loss: 0.4700 - acc: 0.76 - ETA: 10s - loss: 0.4766 - acc: 0.75 - ETA: 10s - loss: 0.4722 - acc: 0.76 - ETA: 10s - loss: 0.4736 - acc: 0.76 - ETA: 10s - loss: 0.4773 - acc: 0.75 - ETA: 10s - loss: 0.4781 - acc: 0.75 - ETA: 10s - loss: 0.4792 - acc: 0.76 - ETA: 9s - loss: 0.4908 - acc: 0.7487 - ETA: 9s - loss: 0.4910 - acc: 0.751 - ETA: 9

128/128 [==============================] - ETA: 13s - loss: 0.4580 - acc: 0.75 - ETA: 13s - loss: 0.4582 - acc: 0.76 - ETA: 13s - loss: 0.4718 - acc: 0.77 - ETA: 12s - loss: 0.4583 - acc: 0.78 - ETA: 12s - loss: 0.4320 - acc: 0.80 - ETA: 12s - loss: 0.4463 - acc: 0.79 - ETA: 12s - loss: 0.4334 - acc: 0.79 - ETA: 12s - loss: 0.4294 - acc: 0.80 - ETA: 12s - loss: 0.4368 - acc: 0.79 - ETA: 12s - loss: 0.4553 - acc: 0.78 - ETA: 11s - loss: 0.4569 - acc: 0.78 - ETA: 11s - loss: 0.4572 - acc: 0.78 - ETA: 11s - loss: 0.4659 - acc: 0.78 - ETA: 11s - loss: 0.4681 - acc: 0.78 - ETA: 11s - loss: 0.4682 - acc: 0.77 - ETA: 11s - loss: 0.4683 - acc: 0.78 - ETA: 11s - loss: 0.4669 - acc: 0.78 - ETA: 11s - loss: 0.4659 - acc: 0.78 - ETA: 11s - loss: 0.4653 - acc: 0.78 - ETA: 11s - loss: 0.4640 - acc: 0.78 - ETA: 11s - loss: 0.4641 - acc: 0.78 - ETA: 10s - loss: 0.4748 - acc: 0.77 - ETA: 10s - loss: 0.4760 - acc: 0.76 - ETA: 10s - loss: 0.4763 - acc: 0.76 - ETA: 10s - loss: 0.4736 - acc: 0.76 - ETA: 10

128/128 [==============================] - ETA: 12s - loss: 0.6978 - acc: 0.62 - ETA: 11s - loss: 0.6097 - acc: 0.68 - ETA: 11s - loss: 0.6216 - acc: 0.69 - ETA: 11s - loss: 0.5716 - acc: 0.72 - ETA: 11s - loss: 0.5973 - acc: 0.70 - ETA: 11s - loss: 0.5720 - acc: 0.72 - ETA: 11s - loss: 0.5581 - acc: 0.73 - ETA: 11s - loss: 0.5592 - acc: 0.72 - ETA: 10s - loss: 0.5697 - acc: 0.70 - ETA: 10s - loss: 0.5520 - acc: 0.72 - ETA: 10s - loss: 0.5551 - acc: 0.71 - ETA: 10s - loss: 0.5410 - acc: 0.72 - ETA: 10s - loss: 0.5309 - acc: 0.73 - ETA: 10s - loss: 0.5306 - acc: 0.73 - ETA: 10s - loss: 0.5184 - acc: 0.74 - ETA: 10s - loss: 0.5229 - acc: 0.74 - ETA: 10s - loss: 0.5167 - acc: 0.74 - ETA: 10s - loss: 0.5107 - acc: 0.74 - ETA: 10s - loss: 0.5099 - acc: 0.74 - ETA: 9s - loss: 0.5091 - acc: 0.7422 - ETA: 9s - loss: 0.5023 - acc: 0.747 - ETA: 9s - loss: 0.5045 - acc: 0.744 - ETA: 9s - loss: 0.5043 - acc: 0.744 - ETA: 9s - loss: 0.5070 - acc: 0.743 - ETA: 9s - loss: 0.5105 - acc: 0.738 - ETA: 9

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigths_file = os.path.join(modelPath, run_name + '.h5')
    print(weigths_file)
    model.save(weigths_file)
saveModel(model, run_name)

In [ ]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)